In [ ]:
import phd

import os
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import nengo
import numpy as np
from scipy import stats
from nengo.utils.matplotlib import rasterplot
from nengo.utils.stdlib import Timer

# Some plotting niceties
phd.plots.setup()

# Ensure TIMIT is extracted
timit_path = "~/phd_data/timit"
timit = phd.timit.TIMIT(timit_path)
try:
    timit.untar(os.path.expanduser("~/Dropbox/LDC93S1.tgz"))
except Exception as e:
    pass

# Some general params
utt = 'deadline'
spkr = 'RRE0'

# NCC

In [ ]:
timit = phd.timit.TIMIT(timit_path)
timit.filefilt.spkr_id = spkr
samples = timit.word_samples([utt])
model = phd.sermo.AuditoryFeatures()
model.audio = samples[utt][0]

In [ ]:
# Get MFCCs
model.mfcc.dt = 0.01
x = model.mfcc()
n_frames = x.shape[0]
print x.shape

# Plot
f, ax1, ax2 = phd.plots.plot_trajs(x, x, zscore=(False, True))
f.set_size_inches(10, 3.4)
f.suptitle("Features for utterance of '%s'" % utt, x=0.44, fontsize='large')
ax1.set_ylabel("MFCC")
ax2.set_ylabel("MFCC (z-scored)")
ax2.set_xlabel("Frame")
f.tight_layout(rect=[0, 0, 1, 0.97])
phd.plots.savefig(f, 'presentation', 'ncc-mfcc')

In [ ]:
# NCCs shortened to 31 frames
net = model.build(nengo.Network(seed=0))
with net:
    ihc_p = nengo.Probe(net.periphery.ihc, synapse=None)
    an_in_p = nengo.Probe(net.periphery.an.input, synapse=None)
    an_p = nengo.Probe(net.periphery.an.add_neuron_output(), synapse=None)
    c_p = nengo.Probe(net.output, synapse=0.01)
sim = nengo.Simulator(net, dt=0.001)
sim.run(model.t_audio)
ncc = phd.experiments.shorten(sim.data[c_p], n_frames=n_frames)

# Plot
t, t_ix = sim.trange(model.t_audio / n_frames), np.arange(0, n_frames, 10)
t_ix[1:] -= 1
f, ax1, ax2 = phd.plots.plot_trajs(ncc, ncc, zscore=(False, True))
f.set_size_inches(10, 3.4)
f.suptitle("Features for utterance of '%s'" % utt, x=0.44, fontsize='large')
ax1.set_ylabel("NCC")
ax2.set_ylabel("NCC (z-scored)")
ax2.set_xticks(t_ix)
ax2.set_xticklabels(np.round(t[t_ix], 3))
ax2.set_xlabel("Time (s)")
f.tight_layout(rect=[0, 0, 1, 0.97])
phd.plots.savefig(f, 'presentation', 'ncc-ncc')

In [ ]:
w = 9.2 * phd.figures.in2px
h = 3.35 * phd.figures.in2px

fig = phd.figures.svgfig(w, h * 2)
fig.append(phd.figures.el(None, phd.figures.svgpath('ncc-mfcc', subdir='presentation'), 0, 0))
fig.append(phd.figures.el(None, phd.figures.svgpath('ncc-ncc', subdir='presentation'), 0, h))
phd.figures.savefig(fig, 'mfcc-ncc', subdir='presentation')

# Syllable production

In [ ]:
path = phd.ges_path('ges-de-cvc', 'ban.ges')
score = phd.vtl.parse_ges(path) 
traj = score.trajectory(dt=0.001)

plt.figure(figsize=(6, 3))
plt.pcolormesh(traj.T, linewidth=0, rasterized=True)
plt.xticks(np.linspace(0, 1200, 6), np.linspace(0, 1, 6))
plt.ylabel("Gesture index")
plt.xlabel("Relative time")
sns.despine()
plt.tight_layout()
plt.savefig(os.path.join(phd.plots.root, 'figures', 'presentation', 'gs-traj.svg'))

# Syllable classification

In [ ]:
with nengo.Network() as model:
    ens_1d = nengo.Ensemble(10, dimensions=1)
sim = nengo.Simulator(model)

plt.figure(figsize=(6, 3))
eval_points, activities = nengo.utils.ensemble.tuning_curves(ens_1d, sim)
plt.plot(eval_points, activities)
plt.ylabel("Firing rate (Hz)")
plt.xlabel("Input scalar, x")
sns.despine()
plt.tight_layout()
plt.savefig(os.path.join(phd.plots.root, 'figures', 'presentation', 'tuning.svg'))

In [ ]:
with nengo.Network() as model:
    ramp = nengo.Node(lambda t: np.clip(t - 0.1, 0, 1))
    ens_1d = nengo.Ensemble(200, dimensions=1)
    nengo.Connection(ramp, ens_1d)
    p = nengo.Probe(ens_1d, synapse=0.01)
sim = nengo.Simulator(model)
sim.run(1.2)

plt.figure(figsize=(6, 3))
plt.plot(sim.trange(), sim.data[p])
plt.xlabel("Time (s)")
sns.despine()
plt.tight_layout()
plt.savefig(os.path.join(phd.plots.root, 'figures', 'presentation', 'ramp.svg'))